# 3일차 1교시 스파크 기본 명령어 이해
> 스파크의 기본 명령어와 구조에 대해 이해합니다

## 목차
* [1. 스파크를 통한 CSV 파일 읽기](#1.-스파크를-통한-CSV-파일-읽기)
* [2. 스파크의 2가지 프로그래밍 방식 비교](#2.-스파크의-2가지-프로그래밍-방식-비교)
* [3. 스파크를 통한 JSON 파일 읽기](#3.-스파크를-통한-JSON-파일-읽기)
* [4. 뷰 테이블 생성 및 조회](#4.-뷰-테이블-생성-및-조회)
* [5. 스파크 애플리케이션의 개념 이해](#5.-스파크-애플리케이션의-개념-이해)
* [6. 스파크 UI](#6.-스파크-UI)
* [7. M&M 초콜렛 분류 예제](#7.-M&M-초콜렛-분류-예제)
* [8. 실습문제](#8.-실습문제)
* [9. 질문과 답변](#9.-질문과-답변)
* 10. 레퍼런스
  * [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
  * <a href="https://spark.apache.org/docs/2.4.5/api/sql/" target="_blank">PySpark 2.4.5 Builtin Functions</a>

## 1. 스파크를 통한 CSV 파일 읽기
> Spark 2.4.5 버전을 기준으로 작성되었습니다. 스파크는 2.0 버전으로 업데이트 되면서 DataFrames 은 Datasets 으로 통합되었고, 기존의 RDD 에서 사용하던 연산 및 기능과 DataFrame 에서 사용하던 것 모두 사용할 수 있습니다. 스파크 데이터 모델은 RDD (Spark1.0) —> Dataframe(Spark1.3) —> Dataset(Spark1.6) 형태로 업그레이드 되었으나, 본문에서 일부 DataFrames 와 DataSets 가 거의 유사하여, 일부 혼용되어 사용되는 경우가 있을 수 있습니다.

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate())

In [26]:
# !which python
!/opt/conda/bin/python --version

print("spark.version: {}".format((spark.version)))
spark

Python 3.8.6
spark.version: 3.0.1


In [27]:
strings = spark.read.text("../requirements.txt")
strings.show(5, truncate=False)
count = strings.count()
print("count of word is {}".format(count))

strings.printSchema()

+--------+
|value   |
+--------+
|boto3   |
|scrapy  |
|selenium|
|mrjob   |
|pyspark |
+--------+
only showing top 5 rows

count of word is 9
root
 |-- value: string (nullable = true)



In [28]:
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.column import Column

assert(type(strings) == DataFrame)
assert(type(strings.value) == Column) # 현재 strings 데이터프레임의 스키마에 value 라는 하나의 컬럼만 존재합니다

In [29]:
help(strings) # 데이터프레임은 Structured API 를 통해 Row 타입의 레코드를 다루는 함수를 이용하고

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(pyspark.sql.pandas.map_ops.PandasMapOpsMixin, pyspark.sql.pandas.conversion.PandasConversionMixin)
 |  DataFrame(jdf, sql_ctx)
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`::
 |  
 |      people = spark.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the :class:`DataFrame`, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SparkSession
 |      people = spark.read.parquet("...")
 |      department = spark.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId ==

In [30]:
help(strings.value) # 컬럼은 컬럼과의 비교 혹은 포함된 문자열을 다루는 contains 같은 함수를 사용합니다

Help on Column in module pyspark.sql.column object:

class Column(builtins.object)
 |  Column(jc)
 |  
 |  A column in a DataFrame.
 |  
 |  :class:`Column` instances can be created by::
 |  
 |      # 1. Select a column out of a DataFrame
 |  
 |      df.colName
 |      df["colName"]
 |  
 |      # 2. Create from an expression
 |      df.colName + 1
 |      1 / df.colName
 |  
 |  .. versionadded:: 1.3
 |  
 |  Methods defined here:
 |  
 |  __add__ = _(self, other)
 |      binary operator
 |  
 |  __and__ = _(self, other)
 |      binary operator
 |  
 |  __bool__ = __nonzero__(self)
 |  
 |  __contains__(self, item)
 |      # container operators
 |  
 |  __div__ = _(self, other)
 |      binary operator
 |  
 |  __eq__ = _(self, other)
 |      binary operator
 |  
 |  __ge__ = _(self, other)
 |      binary operator
 |  
 |  __getattr__(self, item)
 |  
 |  __getitem__(self, k)
 |  
 |  __gt__ = _(self, other)
 |      binary operator
 |  
 |  __init__(self, jc)
 |      Initialize self.

### 아무런 옵션을 주지 않는 경우 스파크가 알아서 컬럼 이름과 데이터 타입을 (string) 지정합니다

In [31]:
log_access = spark.read.csv("data/log_access.csv")
log_access.printSchema()
log_access.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

+----------+-----+------+
|       _c0|  _c1|   _c2|
+----------+-----+------+
|    a_time|a_uid|  a_id|
|1603645200|    1| login|
|1603647200|    1|logout|
|1603649200|    2| login|
|1603650200|    2|logout|
|1603653200|    2| login|
|1603657200|    3| login|
|1603659200|    3|logout|
|1603660200|    4| login|
|1603664200|    4|logout|
|1603664500|    4| login|
|1603666500|    5| login|
|1603669500|    5|logout|
|1603670500|    6| login|
|1603673500|    7| login|
|1603674500|    8| login|
|1603675500|    9| login|
+----------+-----+------+



### 첫 번째 라인에 헤더가 포함되어 있는 경우 아래와 같이 header option 을 지정하면 컬럼 명을 가져올 수 있습니다

In [32]:
log_access = spark.read.option("header", "true").csv("data/log_access.csv")
log_access.printSchema()
log_access.show()

root
 |-- a_time: string (nullable = true)
 |-- a_uid: string (nullable = true)
 |-- a_id: string (nullable = true)

+----------+-----+------+
|    a_time|a_uid|  a_id|
+----------+-----+------+
|1603645200|    1| login|
|1603647200|    1|logout|
|1603649200|    2| login|
|1603650200|    2|logout|
|1603653200|    2| login|
|1603657200|    3| login|
|1603659200|    3|logout|
|1603660200|    4| login|
|1603664200|    4|logout|
|1603664500|    4| login|
|1603666500|    5| login|
|1603669500|    5|logout|
|1603670500|    6| login|
|1603673500|    7| login|
|1603674500|    8| login|
|1603675500|    9| login|
+----------+-----+------+



### inferSchema 옵션으로 데이터 값을 확인하고 스파크가 데이터 타입을 추정하게 할 수 있습니다

In [33]:
log_access = spark.read.option("header", "true").option("inferSchema", "true").csv("data/log_access.csv")
log_access.printSchema()
log_access.show()

root
 |-- a_time: integer (nullable = true)
 |-- a_uid: integer (nullable = true)
 |-- a_id: string (nullable = true)

+----------+-----+------+
|    a_time|a_uid|  a_id|
+----------+-----+------+
|1603645200|    1| login|
|1603647200|    1|logout|
|1603649200|    2| login|
|1603650200|    2|logout|
|1603653200|    2| login|
|1603657200|    3| login|
|1603659200|    3|logout|
|1603660200|    4| login|
|1603664200|    4|logout|
|1603664500|    4| login|
|1603666500|    5| login|
|1603669500|    5|logout|
|1603670500|    6| login|
|1603673500|    7| login|
|1603674500|    8| login|
|1603675500|    9| login|
+----------+-----+------+



## 2. 스파크의 2가지 프로그래밍 방식 비교

### 하나. 구조화된 API 호출을 통해 데이터를 출력하는 방법
> 출력 시에 bigint 값인 날짜는 아래와 같이 from_unixtime 및 to_timestamp 함수를 통해 변환할 수 있습니다.

In [34]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_timestamp, to_date, col, lit

df = spark.read.option("inferSchema", "true").json("data/activity-data")

# 구조화된 API 를 통한 구문
timestamp = df.select(
    "Arrival_Time",
    to_timestamp(from_unixtime(col('Arrival_Time') / lit(1000)), 'yyyy-MM-dd HH:mm:ss').alias('String_Datetime')
)
timestamp.show(5)

+-------------+-------------------+
| Arrival_Time|    String_Datetime|
+-------------+-------------------+
|1424686734992|2015-02-23 19:18:54|
|1424686735190|2015-02-23 19:18:55|
|1424686735395|2015-02-23 19:18:55|
|1424686735593|2015-02-23 19:18:55|
|1424686735795|2015-02-23 19:18:55|
+-------------+-------------------+
only showing top 5 rows



### 둘. 표현식 형식으로 그대로 사용하여 출력하는 방법
> 컬럼(col) 혹은 함수(concat 등)를 직접 사용하는 방식을 **구조화된 API** 를 사용한다고 말하고 SQL 구문으로 표현하는 방식을 **SQL 표현식**을 사용한다고 말합니다

In [35]:
# SQL Expression 통한 구문
ts = df.selectExpr(
    "Arrival_Time",
    "to_timestamp(from_unixtime(Arrival_Time / 1000), 'yyyy-MM-dd HH:mm:ss') as String_Datetime"
)
ts.show(5)

+-------------+-------------------+
| Arrival_Time|    String_Datetime|
+-------------+-------------------+
|1424686734992|2015-02-23 19:18:54|
|1424686735190|2015-02-23 19:18:55|
|1424686735395|2015-02-23 19:18:55|
|1424686735593|2015-02-23 19:18:55|
|1424686735795|2015-02-23 19:18:55|
+-------------+-------------------+
only showing top 5 rows



#### Select 뿐만 아니라 filter 의 경우도 Expression 을 사용할 수 있습니다

In [36]:
df.filter(col("index") > 100).select("index", "user").groupBy("user").count().show()
# 대부분의 구문에서 표현식을 통해 처리할 수 있도록 내부적으로 2가지 방식에 대해 모두 구현되어 있습니다. 
df.filter("index > 100").select("index", "user").groupBy("user").count().show()

+----+-----+
|user|count|
+----+-----+
|   g|91650|
|   f|92030|
|   e|96000|
|   h|77300|
|   d|81220|
|   c|77130|
|   i|92530|
|   b|91210|
|   a|80824|
+----+-----+

+----+-----+
|user|count|
+----+-----+
|   g|91650|
|   f|92030|
|   e|96000|
|   h|77300|
|   d|81220|
|   c|77130|
|   i|92530|
|   b|91210|
|   a|80824|
+----+-----+



## 3. 스파크를 통한 JSON 파일 읽기

In [37]:
json = spark.read.json("data/activity-data")
users = json.filter("index > 100").select("index", "user").groupBy("user").count()
users.show(5)

+----+-----+
|user|count|
+----+-----+
|   g|91650|
|   f|92030|
|   e|96000|
|   h|77300|
|   d|81220|
+----+-----+
only showing top 5 rows



## 4. 뷰 테이블 생성 및 조회
> 이미 생성된 데이터 프레임을 통해서 현재 세션에서만 조회 가능한 임시 뷰 테이블을 만들어 SQL 질의가 가능합니다.

In [38]:
users.createOrReplaceTempView("users")
spark.sql("select * from users where count is not null and count > 9000 order by count desc").show(5)

+----+-----+
|user|count|
+----+-----+
|   e|96000|
|   i|92530|
|   f|92030|
|   g|91650|
|   b|91210|
+----+-----+
only showing top 5 rows



### JSON 파일을 읽는 여러가지 방법

In [39]:
# 스키마 확인 - 3가지 모두 동일한 결과를 얻을 수 있으며 편한 방식을 선택하시면 됩니다
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json") # 미국 교통통계국이 제공하는 항공운항 데이터
df.printSchema()

df2 = spark.read.load("./data/flight-data/json/2015-summary.json", format="json")
df2.printSchema()

df3 = spark.read.json("./data/flight-data/json/2015-summary.json")
df3.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



## 5. 스파크 애플리케이션의 개념 이해

### 스파크에서 반드시 알아야 할 객체와 개념
| 구분 | 설명 | 기타 |
|---|---|---|
| Application | 스파크 프레임워크를 통해 빌드한 프로그램. 전체 작업을 관리하는 Driver 와 Executors 상에서 수행되는 프로그램으로 구분합니다 | - |
| SparkSession | 스파크의 모든 기능을 사용하기 위해 생성하는 객체 | - |
| Job | 하나의 액션(save, collect 등)을 수행하기 위해 여러개의 타스크로 구성된 병렬처리 단위 | DAG 혹은 Spark Execution Plan |
| Stage | 하나의 잡은 다수의 스테이지라는 것으로 구성되며, 하나의 스테이지는 다수의 타스크 들로 구성됩니다 | - |
| Task | 스파크 익스큐터에 보내지는 하나의 작업 단위 | 하나의 Core 혹은 Partition 단위의 작업 |

### 스파크의 변환(Transformation)과 액션(Action)
| 구분 | 설명 | 기타 |
|---|---|---|
| Transformation | 원본 데이터의 변경 없이 새로운 데이터프레임을 생성하는 모든 작업을 말하며 모든 변환 작업들은 lazily evaluated 되며 lineage 를 유지합니다 | select, filter, join, groupBy, orderBy |
| Action | 여태까지 지연된 변환 작업을 트리거링하는 동작을 말하며, 데이터의 조회 및 저장 등의 작업을 말합니다 | show, take, count, collect, save |

> Lineage : 연속된 변환 파이프라인이 액션을 만나기 전까지의 모든 이력 혹은 히스토리 정보를 모두 저장하고 있는 객체를 말하며, 스파크는 이렇게 체인을 구성한 변환 작업을 통해 **쿼리 최적화**를 수행할 수 있으며, 데이터 불변성(Immutability)를 통해서 **내결함성(Fault Tolerance)**을 가질 수 있습니다.

### 좁은 변환과 넓은 변환 - Narrow and Wide Transformation
> 위에서 언급한 최적화(Optimization) 과정은 여러 오퍼레이션들을 다시 여러 스테이지로 쪼개고, 이러한 스테이지 들이 셔플링이 필요한지, 클러스터간의 데이터 교환이 필요한지 등을 결정하는 문제이며, 변환 작업은 크게 하나의 파티션 내에 수행이 가능한 **좁은 의존성(narrow dependencies)** 과 셔플링이 발생하여 클러스터 전체에 데이터 교환이 필요한 **넓은 의존성(wide dependencies)** 두 가지로 구분합니다

![Transformation](images/transformation.png)

## 6. 스파크 UI
> Default 포트는 4040 이므로 http://localhost:4040 에 접속하여 앞에서 배웠던 Narrow, Wide Transformation DAG를 확인합니다

In [40]:
# Narrow Transformation
strings = spark.read.text("../requirements.txt")
jupyter = strings.filter(strings.value.contains("jupyter"))
jupyter.show(truncate=False)

+---------------------------------+
|value                            |
+---------------------------------+
|jupyter_contrib_nbextensions     |
|jupyter_nbextensions_configurator|
+---------------------------------+



In [41]:
# Wide Transformation
user = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_user.csv")
count = user.groupBy("u_gender").count()
count.show(truncate=False)

+--------+-----+
|u_gender|count|
+--------+-----+
|여      |3    |
|남      |6    |
+--------+-----+



| Narrow | Wide |
|---|---|
|![narrow](images/narrow.png)|![wide](images/wide.png)|

## 7. M&M 초콜렛 분류 예제
> [Learning Spark 2nd Edition](https://github.com/psyoblade/LearningSparkV2?organization=psyoblade&organization=psyoblade) 에서 제공하는 data bricks dataset 예제 가운데 미국의 주 별 M&M 초콜렛 판매량을 집계하는 예제를 작성합니다

In [42]:
mnm_df = spark.read.option("header", "true").option("inferSchema", "true").csv("data/databricks/mnm_dataset.csv")
mnm_df.printSchema()
mnm_df.show(truncate=False)

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)

+-----+------+-----+
|State|Color |Count|
+-----+------+-----+
|TX   |Red   |20   |
|NV   |Blue  |66   |
|CO   |Blue  |79   |
|OR   |Blue  |71   |
|WA   |Yellow|93   |
|WY   |Blue  |16   |
|CA   |Yellow|53   |
|WA   |Green |60   |
|OR   |Green |71   |
|TX   |Green |68   |
|NV   |Green |59   |
|AZ   |Brown |95   |
|WA   |Yellow|20   |
|AZ   |Blue  |75   |
|OR   |Brown |72   |
|NV   |Red   |98   |
|WY   |Orange|45   |
|CO   |Blue  |52   |
|TX   |Brown |94   |
|CO   |Red   |82   |
+-----+------+-----+
only showing top 20 rows



In [43]:
from pyspark.sql.functions import *

# We use the DataFrame high-level APIs. Note
# that we don't use RDDs at all. Because some of Spark's
# functions return the same object, we can chain function calls.
# 1. Select from the DataFrame the fields "State", "Color", and "Count"
# 2. Since we want to group each state and its M&M color count,
# we use groupBy()
# 3. Aggregate counts of all colors and groupBy() State and Color
# 4 orderBy() in descending order
count_mnm_df = (mnm_df.select("State", "Color", "Count") \
.groupBy("State", "Color") \
.agg(count("Count").alias("Total")) \
.orderBy("Total", ascending=False))
# Show the resulting aggregations for all the states and colors;
# a total count of each color per state.
# Note show() is an action, which will trigger the above
# query to be executed.
count_mnm_df.show(n=60, truncate=False)
print("Total Rows = %d" % (count_mnm_df.count()))

# While the above code aggregated and counted for all
# the states, what if we just want to see the data for
# a single state, e.g., CA?
# 1. Select from all rows in the DataFrame
# 2. Filter only CA state
# 3. groupBy() State and Color as we did above
# 4. Aggregate the counts for each color
# 5. orderBy() in descending order
# Find the aggregate count for California by filtering
ca_count_mnm_df = (mnm_df.select("State", "Color", "Count") \
.where(mnm_df.State == "CA") \
.groupBy("State", "Color") \
.agg(count("Count").alias("Total")) \
.orderBy("Total", ascending=False))
# Show the resulting aggregation for California.
# As above, show() is an action that will trigger the execution of the
# entire computation.
ca_count_mnm_df.show(n=10, truncate=False)
# Stop the SparkSession
# spark.stop()

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|AZ   |Brown |1698 |
|CO   |Blue  |1695 |
|WY   |Green |1695 |
|NM   |Red   |1690 |
|AZ   |Orange|1689 |
|NM   |Yellow|1688 |
|NM   |Brown |1687 |
|UT   |Orange|1684 |
|NM   |Green |1682 |
|UT   |Red   |1680 |
|AZ   |Green |1676 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WY   |Red   |1670 |
|WA   |Brown |1669 |
|NM   |Orange|1665 |
|WY   |Blue  |1664 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|NV   |Brown |1657 |
|CA   |Orange|1657 |
|CO   |Brown |1656 |
|CA   |Red   |1656 |
|UT   |Blue  |1655 |
|AZ   |Yellow|1654 |
|TX   |Orange|1652 |
|AZ   |Red   |1648 |
|OR   |Blue  |1646 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|CO   |Orange|1642 |
|TX   |Brown 

## 8. 실습문제

### 실습#1. data/tbl_user.csv 파일을 읽고, 올바른 데이터 타입을 추정할 수 있도록 옵션을 주어 데이터를 읽고, 해당 스키마를 출력하세요
> 참고 : printSchema()

In [44]:
user = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_user.csv")
user.printSchema()

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)



### 실습#2. data/tbl_purchase.csv 파일을 읽고, 데이터를 출력하되 p_time 필드는 날짜 함수를 이용하여 식별 가능하도록 데이터를 출력하세요
> 참고 : from_unixtime(column), show()

In [45]:
purchase = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_purchase.csv")
purchase.selectExpr("*", "from_unixtime(p_time)").show(truncate=False)

+----------+-----+----+-----------+--------+----------------------------------------------------------+
|p_time    |p_uid|p_id|p_name     |p_amount|from_unixtime(CAST(p_time AS BIGINT), yyyy-MM-dd HH:mm:ss)|
+----------+-----+----+-----------+--------+----------------------------------------------------------+
|1603651550|1    |2000|LG DIOS    |2000000 |2020-10-26 03:45:50                                       |
|1603694755|1    |2000|LG Gram    |1800000 |2020-10-26 15:45:55                                       |
|1603673500|2    |2001|LG Cyon    |1400000 |2020-10-26 09:51:40                                       |
|1603652155|3    |2002|LG TV      |1000000 |2020-10-26 03:55:55                                       |
|1603674500|4    |2003|LG Computer|4500000 |2020-10-26 10:08:20                                       |
|1603665955|5    |2004|LG Gram    |3500000 |2020-10-26 07:45:55                                       |
|1603666155|5    |2004|LG TV      |2500000 |2020-10-26 07:49:15 

## 9. 질문과 답변

### Q1. 하나의 물리적인 장비 혹은 JVM 위에서 여러개의 SparkSession 실행이 가능한가?
> [A tale of Spark Session and Spark Context](https://medium.com/@achilleus/spark-session-10d0d66d1d24) 페이지를 통해 다수 세션을 생성하는 예제를 통해 이해할 수 있습니다. Spark 2.0 이전에는 Spark Context, Hive Context, SQL Context 등 다양한 기능을 별도의 컨텍스트 객체를 통해 사용할 수 있었지만, 2.0 이후부터는 SparkSession 이라는 통합된 객체를 통해 접근 및 사용할 수 있습니다.

### A1. Spark Session builder 옵션을 통해 하나의 JVM 에서 다수의 컨텍스트 생성은 가능하지만, 좋지 않은 방법
  - enableHiveSuppoort : 기존의 [SQLContext, HiveContext](https://spark.apache.org/docs/3.0.0-preview/sql-migration-guide.html#upgrading-from-spark-sql-24-to-30) 대신 catalog 인터페이스를 통해 Metastore, SerDe 및 UDFs 등을 사용을 활성화 하며, 테이블 생성 및 조회가 가능합니다
  - getOrCreate : 이미 존재한다면 기존의 객체를 사용하고, 그렇지 않은 경우에만 생성합니다
* 하나의 스파크 노트북 환경을 다수의 유저가 사용
  - "spark.driver.allowMultipleContexts" 설정을 true 로 하여 가능하지만, 하나의 JVM 상에서 여러개의 Spark Context 를 유지하는 것은 위험할 수 있습니다

### A1. SparkSession.newSession() 명령을 통해 SparkContext 는 동일하되, 설정정보만 별도로 유지할 수 있습니다
  - 아래와 같이 2개의 세션을 생성하고, 개별 conf 설정을 통해 2가지 세션을 유지할 수 있습니다.

In [46]:
help(spark)

Help on SparkSession in module pyspark.sql.session object:

class SparkSession(pyspark.sql.pandas.conversion.SparkConversionMixin)
 |  SparkSession(sparkContext, jsparkSession=None)
 |  
 |  The entry point to programming Spark with the Dataset and DataFrame API.
 |  
 |  A SparkSession can be used create :class:`DataFrame`, register :class:`DataFrame` as
 |  tables, execute SQL over tables, cache tables, and read parquet files.
 |  To create a SparkSession, use the following builder pattern:
 |  
 |  >>> spark = SparkSession.builder \
 |  ...     .master("local") \
 |  ...     .appName("Word Count") \
 |  ...     .config("spark.some.config.option", "some-value") \
 |  ...     .getOrCreate()
 |  
 |  .. autoattribute:: builder
 |     :annotation:
 |  
 |  Method resolution order:
 |      SparkSession
 |      pyspark.sql.pandas.conversion.SparkConversionMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |      Enable 'with SparkSession.builder.(...).g

In [47]:
help(spark.catalog)

Help on Catalog in module pyspark.sql.catalog object:

class Catalog(builtins.object)
 |  Catalog(sparkSession)
 |  
 |  User-facing catalog API, accessible through `SparkSession.catalog`.
 |  
 |  This is a thin wrapper around its Scala implementation org.apache.spark.sql.catalog.Catalog.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sparkSession)
 |      Create a new Catalog that wraps the underlying JVM object.
 |  
 |  cacheTable(self, tableName)
 |      Caches the specified table in-memory.
 |      
 |      .. versionadded:: 2.0
 |  
 |  clearCache(self)
 |      Removes all cached tables from the in-memory cache.
 |      
 |      .. versionadded:: 2.0
 |  
 |  createExternalTable(self, tableName, path=None, source=None, schema=None, **options)
 |      Creates a table based on the dataset in a data source.
 |      
 |      It returns the DataFrame associated with the external table.
 |      
 |      The data source is specified by the ``source`` and a set of ``options``.
 

In [48]:
spark2 = spark.newSession()
print(spark)
print(spark2)
assert(spark != spark2)

print(spark.sparkContext)
print(spark2.sparkContext)
assert(spark.sparkContext == spark2.sparkContext)

<SparkContext master=local[*] appName=pyspark-shell>
<SparkContext master=local[*] appName=pyspark-shell>


In [49]:
def printConfiguration(spark):
    defaultShufflePartitions = int(spark.conf.get("spark.sql.shuffle.partitions"))
    defaultMinPartitions = spark.sparkContext.defaultMinPartitions
    defaultParallelism = spark.sparkContext.defaultParallelism
    defaultMaxPartitionBytes = spark.conf.get("spark.sql.files.maxPartitionBytes")
    defaultOpenCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes"))
    sqlCrossJoinEnabled = spark.conf.get("spark.sql.crossJoin.enabled")
    sqlWarehouseDir = spark.conf.get("spark.sql.warehouse.dir")

    print("defaultShufflePartitions:{}".format(defaultShufflePartitions))
    print("defaultMinPartitions:{}".format(defaultMinPartitions))
    print("defaultParallelism:{}".format(defaultParallelism))
    print("defaultMaxPartitionBytes:{}".format(defaultMaxPartitionBytes))
    print("defaultOpenCostInBytes:{}".format(defaultOpenCostInBytes))
    print("sqlCrossJoinEnabled:{}".format(sqlCrossJoinEnabled))
    print("sqlWarehouseDir:{}".format(sqlWarehouseDir))

In [50]:
mnm_df = spark.read.option("header", "true").option("inferSchema", "true").csv("data/databricks/mnm_dataset.csv")
mnm_df.createOrReplaceTempView("mnm_dataset")
spark.sql("show tables").show()
spark.catalog.listTables()
spark.conf.get("spark.sql.crossJoin.enabled")
spark.conf.set("spark.sql.shuffle.partitions", 100)
spark.sparkContext.getConf().getAll()
printConfiguration(spark)

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
|        |mnm_dataset|       true|
|        |      users|       true|
+--------+-----------+-----------+

defaultShufflePartitions:100
defaultMinPartitions:2
defaultParallelism:4
defaultMaxPartitionBytes:134217728b
defaultOpenCostInBytes:4194304
sqlCrossJoinEnabled:true
sqlWarehouseDir:file:/home/jovyan/work/spark-warehouse


In [51]:
spark2.sql("show tables").show()
spark2.catalog.listTables()
spark2.conf.get("spark.sql.crossJoin.enabled")
spark2.sparkContext.getConf().getAll()
printConfiguration(spark2)

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

defaultShufflePartitions:200
defaultMinPartitions:2
defaultParallelism:4
defaultMaxPartitionBytes:134217728b
defaultOpenCostInBytes:4194304
sqlCrossJoinEnabled:true
sqlWarehouseDir:file:/home/jovyan/work/spark-warehouse


### Q2. 코드를 통해서 데이터 프레임을 임의로 생성할 수 있나요?

### A2. Scala 는 Seq 객체를 통해서, Python 은 Dict와 Row 객체를 통해서 생성이 가능합니다
* Scala Example
```scala
val emp = Seq((101, "Amy", Some(2)))
val employee = spark.createDataFrame(emp).toDF("employeeId","employeeName","managerId")
```
* Python Example
```python
data = {'employeeId':101, 'employeeName':'Amy', 'managerId':2}
dataRDD = spark.sparkContext.parallelize(Row(data))
```

In [59]:
from pyspark.sql import Row
# emp = {'employeeId':101, 'employeeName':'Amy', 'managerId':2} # inferSchema 오류가 발생하므로 명시적인 스키마 지정이 필요합니다
emp = [Row(employeeId=1, employeeName='Severin', managerId=0), Row(employeeId=2, employeeName='John', managerId=1)]
employee = spark.createDataFrame(spark.sparkContext.parallelize(emp))
employee.printSchema()
employee.show(truncate=False)

root
 |-- employeeId: long (nullable = true)
 |-- employeeName: string (nullable = true)
 |-- managerId: long (nullable = true)

+----------+------------+---------+
|employeeId|employeeName|managerId|
+----------+------------+---------+
|1         |Severin     |0        |
|2         |John        |1        |
+----------+------------+---------+



In [54]:
from pyspark.sql import Row
data = {'visitor': ['foo', 'bar', 'jelmer'], 
        'A': [0, 1, 0],
        'B': [1, 0, 1],
        'C': [1, 0, 0]}

df = spark.sparkContext.parallelize(Row(data))
ddf = spark.createDataFrame(df)
ddf.printSchema()
ddf.show(truncate=False)

root
 |-- A: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- B: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- C: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- visitor: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---------+---------+---------+------------------+
|A        |B        |C        |visitor           |
+---------+---------+---------+------------------+
|[0, 1, 0]|[1, 0, 1]|[1, 0, 0]|[foo, bar, jelmer]|
+---------+---------+---------+------------------+

